In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import json
import urllib
from requests import get
from pandas import json_normalize

In [ ]:
path = '/content/drive/MyDrive/Degree Project/dataset/csv_format/'
extracts = '/content/drive/MyDrive/Degree Project/dataset/extracts/'
elevation_path = '/content/drive/MyDrive/Degree Project/dataset/extracts/elevation_profiles'
local_path = '/content/'
shape_df = pd.read_csv(path + 'shapes.csv')

In [ ]:
shape_data = shape_df.groupby(['shape_id'])
shape_data = shape_data.first().append(shape_data.last()).sort_index().reset_index().drop_duplicates()
shape_data = shape_data.sort_values(['shape_id', 'shape_pt_sequence'], ascending = True)
shape_data = shape_data.drop(shape_data[shape_data.shape_id == 3027].index).reset_index(drop=True) # dropping redundant shapes
shape_data.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,3096,48.42144,-89.26211,0,0
1,3096,48.43579,-89.21709,108,8690
2,3100,48.40297,-89.26967,0,0
3,3100,48.38246,-89.24580,121,7887
4,3104,48.36516,-89.28197,0,0


In [ ]:
shape_ids = list(shape_data.shape_id.unique())

In [ ]:
P1 = shape_data[['shape_pt_lat', 'shape_pt_lon']][::2].to_numpy().tolist()
P2 = shape_data[['shape_pt_lat', 'shape_pt_lon']][1::2].to_numpy().tolist()
h_dist = (shape_data['shape_dist_traveled'][1::2] * 0.001).tolist()

In [ ]:
def get_elevation(lat, lon):
  '''
      script for returning elevation in m from lat, long
  '''
  if lat is None or lon is None: return None
  query = ('https://api.airmap.com/elevation/v1/ele/?points={},{}'.format(lat, lon))
    
  # Request with a timeout for slow responses
  r = get(query, timeout = 20)

  # Only get the json response in case of 200 or 201
  if r.status_code == 200 or r.status_code == 201:
    elevation = json_normalize(r.json())['data'].values[0]
  else:
    elevation = None
  try:
    return elevation[0] # m
  except:
    return elevation

In [ ]:
get_elevation(48.43579000000028, 48.47870000000009)

-8

In [ ]:
#HAVERSINE FUNCTION
def haversine(lat1,lon1,lat2,lon2):
    lat1_rad=math.radians(lat1)
    lat2_rad=math.radians(lat2)
    lon1_rad=math.radians(lon1)
    lon2_rad=math.radians(lon2)
    delta_lat=lat2_rad-lat1_rad
    delta_lon=lon2_rad-lon1_rad
    a=math.sqrt((math.sin(delta_lat/2))**2+math.cos(lat1_rad)*math.cos(lat2_rad)*(math.sin(delta_lon/2))**2)
    d=2*6371000*math.asin(a)
    return d

In [ ]:
rg_data = {}
rg_data['shape_id'] = []
rg_data['road_grade(%)'] = []
for p1, p2, dist, shape_id in zip(P1, P2, h_dist, shape_ids):
  rg_data['shape_id'].append(shape_id)
  s = 100 # number of points between points
  interval_lat = (p2[0] - p1[0])/s # interval for latitude
  interval_lon = (p2[1] - p1[1])/s # interval for longitude
  # set new variables for start point
  lat0 = p1[0]
  lon0 = p1[0]
  # list of latitude and longitude
  lat_list = [lat0]
  lon_list = [lon0]
  # generate points
  for i in range(s):
    lat_step=lat0+interval_lat
    lon_step=lon0+interval_lon
    lon0=lon_step
    lat0=lat_step
    lat_list.append(lat_step)
    lon_list.append(lon_step)
  # distance, elevation, and road grade CALCULATION
  d_list, elev_list, rg_list =[], [], [] # distance, elevation, and road grade list
  for j in range(len(lat_list)):
      lat_p=lat_list[j]
      lon_p=lon_list[j]
      dp=haversine(lat0,lon0,lat_p,lon_p)/1000 #km
      elev = get_elevation(lat_p, lon_p) # m
      d_list.append(dp)
      elev_list.append(elev)
  d_list_rev=d_list[::-1] #reverse list
  mean_elev = round((sum(elev_list)/len(elev_list)), 3)
  min_elev = min(elev_list)
  max_elev = max(elev_list)
  distance = d_list_rev[-1]
  for i in range(len(d_list_rev)):
    try:
      rg = (((elev_list[i] + min_elev)*0.001)/d_list_rev[i]) * 100 # road grade %
    except:
      rg = 0
    rg_list.append(rg)
  rg_data['road_grade(%)'].append(round((sum(rg_list)/len(rg_list)), 3))
  if not os.path.exists(elevation_path):
    os.mkdir(elevation_path)
    # plot and save elevation profile
    image_format = '{}_elevation_profile.png'.format(shape_id)
    base_reg=0
    plt.figure(figsize=(10,4))
    plt.plot(d_list_rev,elev_list)
    plt.plot([0,distance],[min_elev,min_elev],'--g',label='min: '+str(min_elev)+' m')
    plt.plot([0,distance],[max_elev,max_elev],'--r',label='max: '+str(max_elev)+' m')
    plt.plot([0,distance],[mean_elev,mean_elev],'--y',label='ave: '+str(mean_elev)+' m')
    plt.fill_between(d_list_rev,elev_list,base_reg,alpha=0.1)
    plt.text(d_list_rev[0],elev_list[0],"P1")
    plt.text(d_list_rev[-1],elev_list[-1],"P2")
    plt.xlabel("Distance(km)")
    plt.ylabel("Elevation(m)")
    plt.grid()
    plt.legend(fontsize='small')
    plt.ioff()
    plt.savefig(image_format)
    shutil.move(local_path + image_format, elevation_path + '/' + image_format)

In [ ]:
#rg_data = {}
#rg_data['shape_id'] = []
#rg_data['road_grade(%)'] = []
#for p1, p2, dist, shape_id in zip(P1, P2, h_dist, shape_ids): # for every start and stop...get the road grade
  #rg_data['shape_id'].append(shape_id)
  #try:
    #rg_data['road_grade(%)'].append(((get_elevation(p2)-get_elevation(p1))/dist)*100)
  #except:
    #rg_data['road_grade(%)'].append('NA')

In [ ]:
rg_data = pd.DataFrame.from_dict(rg_data, orient='index').transpose()
rg_data.head()

,shape_id,road_grade(%)
0,3096.0,-2.429
1,3100.0,-3.604
2,3104.0,0.000
3,3111.0,-3.604
4,3112.0,-3.604


In [ ]:
rg_data.to_csv('road_grades.csv', index = False)
shutil.move(local_path + 'road_grades.csv', extracts + 'road_grades.csv');